In [ ]:
from roboflow import Roboflow
import os, glob
from visualization import show_random_images_with_bboxes

def download_dataset(api_key: str, workspace: str, project: str, version: int):
    rf = Roboflow(api_key=api_key)
    dataset = rf.workspace(workspace).project(project).version(version).download("yolov8")
    return dataset

dataset = download_dataset(api_key="K0xg5GEEinqPgaqjKKzz", workspace="matyworkspace", project="damagedhealthytrafficsigns", version=9)

image_dir = os.path.join(dataset.location, "test", "images")
label_dir = os.path.join(dataset.location, "test", "labels")
image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))

#show_random_images_with_bboxes(image_paths, label_dir, N=8, cols=4)

In [ ]:
#blur_levels = [0, 0.5, 1.0, 2.0, 4.0]
blur_levels = [0.1, 0.2, 0.35]

In [ ]:
from image_modification import create_property_groups_relative
import json

output_base_dir = dataset.location
blur_groups = create_property_groups_relative(output_base_dir, image_paths, 'blur', blur_levels)


with open("blur_groups.json", "w", encoding="utf-8") as f:
    json.dump(blur_groups, f, ensure_ascii=False, indent=2)

In [ ]:
import json

blur_groups = json.load(open("blur_groups.json", "r", encoding="utf-8"))

In [ ]:
from get_predictions import save_predictions_to_file
import os

for group_name, group_data in blur_groups.items():
    print(f"Group: {group_name}, Number of images: {len(group_data['images'])}")
    predictions_file = f"predictions_blur_{group_name}.json"
    if not os.path.exists(predictions_file):
        save_predictions_to_file([img['modified_path'] for img in group_data['images']], predictions_file)

In [ ]:
from process_predictions import process_all_predictions
import os

for group_name, group_data in blur_groups.items():
    predictions_file = f"predictions_blur_{group_name}.json"
    processed_predictions_file = f"processed_predictions_blur_{group_name}.json"
    if not os.path.exists(processed_predictions_file):
        image_paths = [img['modified_path'] for img in group_data['images']]
        process_all_predictions(image_paths, predictions_file, label_dir, processed_predictions_file)

In [ ]:
import pandas as pd
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
from group_metrics_calculation import calculate_metrics_for_group_relative
for group_name, group_data in blur_groups.items():
    processed_predictions_file = f"processed_predictions_blur_{group_name}.json"
    with open(processed_predictions_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    blur_groups[group_name]['metrics'] = calculate_metrics_for_group_relative(df)
    print(f"Metrics for group {group_name}: {blur_groups[group_name]['metrics']}")

    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

property_name = 'blur'
metrics_to_plot = ['precision_healthy', 'precision_damaged', 'recall_healthy', 'recall_damaged', 
                   'f1_healthy', 'f1_damaged', 'mAP_50', 'mAP_50_95']
metric_labels = ['Precision Healthy', 'Precision Damaged', 'Recall Healthy', 'Recall Damaged', 
                 'F1 Healthy', 'F1 Damaged', 'mAP@0.5', 'mAP@0.5:0.95']

property_values = []
metrics_data = {metric: [] for metric in metrics_to_plot}
group_names = []

for group_name, group_data in blur_groups.items():
    property_values.append(group_data['relative_change'])
    group_names.append(group_name)
    
    for metric in metrics_to_plot:
        metrics_data[metric].append(group_data['metrics'][metric])

sorted_indices = np.argsort(property_values)
property_values = np.array(property_values)[sorted_indices]
group_names = np.array(group_names)[sorted_indices]

for i, (metric, label) in enumerate(zip(metrics_to_plot, metric_labels)):
    metric_values = np.array(metrics_data[metric])[sorted_indices]
    
    plt.figure(figsize=(10, 6))
    plt.errorbar(property_values, metric_values, 
                 fmt='o-', capsize=5, capthick=2, linewidth=1)
    
    for j, (x, y) in enumerate(zip(property_values, metric_values)):
        plt.annotate(f'{y:.3f}', (x, y), textcoords="offset points", xytext=(0,15), 
                    ha='center', fontsize=10, fontweight='bold')
    
    y_margin = (metric_values.max() - metric_values.min()) * 0.15
    plt.ylim(metric_values.min() - y_margin, metric_values.max() + y_margin)
    
    plt.xlabel(property_name.title())
    plt.ylabel(label)
    plt.title(f'{label} vs {property_name.title()}')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
import json
from models import ObjectPrediction
PROPERTY = 'blur'
GROUPS = ['blur_rel_+0.10', 'blur_rel_+0.20', 'blur_rel_+0.35', 'blur_rel_+0.50', 'blur_rel_+1.00', 'blur_rel_+2.00', 'blur_rel_+4.00']

ppredictions_original_file = "none.json"
ppredictions_original: dict[str, list[ObjectPrediction]] = {}

with open(ppredictions_original_file, "r", encoding="utf-8") as f:
    for item in json.load(f):
        image_path = item['image']
        if image_path not in ppredictions_original:
            ppredictions_original[image_path] = []
        ppredictions_original[image_path].append(ObjectPrediction.from_dict(item))

blur_groups = json.load(open("blur_groups.json", "r", encoding="utf-8"))
combined_predictions = {}

for group in GROUPS:    

    ppredictions_modified_file = f"processed_predictions_{PROPERTY}_{group}.json"
    ppredictions_modified: dict[str, list[ObjectPrediction]] = {}

    with open(ppredictions_modified_file, "r", encoding="utf-8") as f:
        for item in json.load(f):
            image_path = item['image']
            if image_path not in ppredictions_modified:
                ppredictions_modified[image_path] = []
            ppredictions_modified[image_path].append(ObjectPrediction.from_dict(item))
    
    blur_group = blur_groups[group]

    for img in blur_group['images']:
        original_path = img['original_path']
        modified_path = img['modified_path']
        orig_preds = ppredictions_original.get(original_path, [])
        mod_preds = ppredictions_modified.get(modified_path, [])
        objects = {}
        for pred in orig_preds:
            key = pred.actual_bbox.__str__() if pred.actual_bbox is not None else pred.predicted_bbox.__str__()
            objects[key] = {
                'gt': (pred.actual, pred.actual_bbox.to_dict() if pred.actual_bbox else None),
                'original_pred': (pred.predicted_label, pred.predicted_bbox.to_dict() if pred.predicted_bbox else None),
            }
        for pred in mod_preds:
            key = pred.actual_bbox.__str__() if pred.actual_bbox is not None else pred.predicted_bbox.__str__()
            if key in objects:
                objects[key]['modified_pred'] = (pred.predicted_label, pred.predicted_bbox.to_dict() if pred.predicted_bbox else None)
            else:
                objects[key] = {
                    'gt': (pred.actual, pred.actual_bbox.to_dict() if pred.actual_bbox else None),
                    'modified_pred': (pred.predicted_label, pred.predicted_bbox.to_dict() if pred.predicted_bbox else None),
                }
        image_combined_predictions = {
            'original_path': original_path,
            'modified_path': modified_path,
            'objects': list(objects.values()),
            'property_change': blur_group['relative_change']
        }
        original_false_detections = 0
        modified_false_detections = 0
        reduced_false_backgrounds = 0
        added_false_backgrounds = 0
        reduced_misclassifications = 0
        added_misclassifications = 0
        for obj in image_combined_predictions['objects']:
            gt_label = obj['gt'][0]
            original_label = obj.get('original_pred', ('background', None))[0]
            modified_label = obj.get('modified_pred', ('background', None))[0]
            if gt_label == 'background':
                if original_label != 'background':
                    original_false_detections += 1
                if modified_label != 'background':
                    modified_false_detections += 1
            else:
                if original_label == 'background' and modified_label == gt_label:
                    reduced_false_backgrounds += 1
                if original_label == gt_label and modified_label == 'background':
                    added_false_backgrounds += 1
                if original_label != gt_label and original_label != 'background' and modified_label == gt_label:
                    reduced_misclassifications += 1
                if original_label == gt_label and modified_label != gt_label and modified_label != 'background':
                    added_misclassifications += 1
        combined_predictions[f"{blur_group['relative_change']}_{original_path}"] = {
            **image_combined_predictions,
            'original_false_detections': original_false_detections,
            'modified_false_detections': modified_false_detections,
            'reduced_false_backgrounds': reduced_false_backgrounds,
            'added_false_backgrounds': added_false_backgrounds,
            'reduced_misclassifications': reduced_misclassifications,
            'added_misclassifications': added_misclassifications,
        }

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

dfc = pd.DataFrame(combined_predictions).T

grouped_data = dfc.groupby('property_change').agg({
    'original_false_detections': 'sum',
    'modified_false_detections': 'sum',
    'reduced_false_backgrounds': 'sum',
    'added_false_backgrounds': 'sum',
    'reduced_misclassifications': 'sum',
    'added_misclassifications': 'sum',
}).reset_index()


In [ ]:
def plot_data_2(grouped_data, title, y_title, neg_key, pos_key, pos_color='green'):
    # Create the bar plot
    fig, ax = plt.subplots(figsize=(12, 6))

    x = np.arange(len(grouped_data) + 1)
    width = 0.35

    bar_keys = ['0'] + list(grouped_data['property_change'])
    bar_heights = [grouped_data[neg_key][0]] + [x for x in grouped_data[pos_key]]


    bars1 = ax.bar(x, bar_heights, width, 
                alpha=0.8, color=pos_color)

    # Customize the plot
    ax.set_xlabel('Blur')
    ax.set_ylabel(y_title)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(bar_keys)
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Add value labels on bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                ax.annotate(f'{int(height)}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=9)

    add_value_labels(bars1)

    plt.tight_layout()
    plt.show()

def plot_grouped_data(grouped_data, title, y_title, neg_key, pos_key, neg_label, pos_label, neg_color='red', pos_color='green'):
    # Create the bar plot
    fig, ax = plt.subplots(figsize=(12, 6))

    x = np.arange(len(grouped_data))
    width = 0.35

    bars1 = ax.bar(x - width/2, grouped_data[neg_key], width, 
                label=neg_label, alpha=0.8, color=neg_color)
    bars2 = ax.bar(x + width/2, grouped_data[pos_key], width, 
                label=pos_label, alpha=0.8, color=pos_color)

    # Customize the plot
    ax.set_xlabel('Blur')
    ax.set_ylabel(y_title)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(grouped_data['property_change'])
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Add value labels on bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                ax.annotate(f'{int(height)}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=9)

    add_value_labels(bars1)
    add_value_labels(bars2)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_data_2(grouped_data, 
                  title='False Detections vs Blur', 
                  y_title='Total False Detections',
                  neg_key='original_false_detections', 
                  pos_key='modified_false_detections', 
                  pos_color='blue')

plot_grouped_data(grouped_data,
                title='Change in False Backgrounds vs Blur', 
                y_title='False Backgrounds',
                neg_key='added_false_backgrounds', 
                pos_key='reduced_false_backgrounds', 
                neg_label='Added False Backgrounds', 
                pos_label='Reduced False Backgrounds', 
                neg_color='red', 
                pos_color='green')

plot_grouped_data(grouped_data,
                title='Change in Misclassifications vs Blur', 
                y_title='Misclassifications',
                neg_key='added_misclassifications', 
                pos_key='reduced_misclassifications', 
                neg_label='Added Misclassifications', 
                pos_label='Reduced Misclassifications', 
                neg_color='red', 
                pos_color='green')